# Tutorial 2: Custom Training YOLO Model

In [ ]:
import cv2
import matplotlib.pyplot as plt
from yolo.tools import XMLtoInputConvertor
from yolo.data import YOLODatasetGenerator
from yolo.estimator import YOLOEstimator
from yolo.detector import YOLOObjectDetector

## Convert XML anotations to YOLO input format

In [ ]:
IC = XMLtoInputConvertor()

IC.convert(XML_dir='example_trainset/apple.v1-apple-dataset.voc/train', 
           annotation_path='example_trainset/apple.v1-apple-dataset.voc/processed_anotation/train.txt')

IC.convert(XML_dir='example_trainset/apple.v1-apple-dataset.voc/valid', 
           annotation_path='example_trainset/apple.v1-apple-dataset.voc/processed_anotation/valid.txt')

IC.export_class_names(class_names_path='example_trainset/apple.v1-apple-dataset.voc/processed_anotation/class.txt')

## Read data

In [ ]:
version = 'v4_tiny'

train_data = YOLODatasetGenerator(anotation_path='example_trainset/apple.v1-apple-dataset.voc/processed_anotation/train.txt',
                                  version=version,
                                  input_sizes=416,
                                  batch_size=4,
                                  data_aug=False,
                                  load_images_to_ram=True,
                                  anchor_per_scale=3,
                                  max_bbox_per_scale=100)

valid_data = YOLODatasetGenerator(anotation_path='example_trainset/apple.v1-apple-dataset.voc/processed_anotation/valid.txt',
                                 version=version,
                                 input_sizes=416,
                                 batch_size=3,
                                 data_aug=False,
                                 load_images_to_ram=True,
                                 anchor_per_scale=3,
                                 max_bbox_per_scale=100)


## Create Model

In [ ]:
yolo_custom_estimator = YOLOEstimator(version=version, 
                                      input_size=416, 
                                      channels=3, 
                                      inference_using_pretrained=False, 
                                      init_pretrained_weight=True, 
                                      disable_gpu=False,
                                      pretrained_weight_dir='./pretrained_weight/')

## Train Model

In [ ]:
yolo_custom_estimator.fit(train_data=train_data,
                          val_data=valid_data,
                          epocs=20,
                          warmup_epocs=5,
                          save_checkpoint=False,
                          save_best_only=True)
yolo_custom_estimator.save(model_dir='custom_model')

## Make Inference

In [ ]:
yolo_custom_detector = YOLOObjectDetector(model=yolo_custom_estimator, 
                                          classes=['apple'])

image_path = 'example_trainset/apple.v1-apple-dataset.voc/valid/AnyConv-com__38_jpg.rf.efc76c3d7c82233abb99ad2c7bcc2f05.jpg'
image_rgb = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)

result, bboxes = yolo_custom_detector.detect_image(image_rgb=image_rgb, 
                                                   score_threshold=0.5, 
                                                   iou_threshold=0.5,
                                                   show_label=True,
                                                   show_confidence=True)
plt.imshow(result)